In [ ]:
# Load Dataset
df = pd.read_csv("workout-dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           70000 non-null  int64  
 1   height                        70000 non-null  int64  
 2   weight                        70000 non-null  int64  
 3   fitness_goals                 70000 non-null  object 
 4   fitness_level                 70000 non-null  object 
 5   health_conditions             68600 non-null  object 
 6   BMI                           70000 non-null  float64
 7   Preferred Workout Duration    70000 non-null  int64  
 8   Joint Issues                  70000 non-null  object 
 9   Mobility & Flexibility Score  70000 non-null  int64  
 10  Daily Activity Type           70000 non-null  object 
 11  Favorite Workout Types        70000 non-null  object 
 12  Fatigue Level                 70000 non-null  object 
 13  r

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

# Load the data
df = pd.read_csv('workout-dataset.csv')

# 1. Data Preprocessing
features = ['age', 'height', 'weight', 'fitness_goals', 'fitness_level',
           'health_conditions', 'BMI', 'Preferred Workout Duration',
           'Joint Issues', 'Mobility & Flexibility Score',
           'Daily Activity Type', 'Favorite Workout Types', 'Fatigue Level']

targets = ['recommended_workout', 'recommended_sets', 'recommended_repetitions']

# Replace nan with 'None' string in health_conditions
df['health_conditions'] = df['health_conditions'].fillna('None')

# 2. Encoding categorical variables
le_dict = {}
categorical_cols = ['fitness_goals', 'fitness_level', 'health_conditions',
                   'Joint Issues', 'Daily Activity Type', 'Favorite Workout Types',
                   'Fatigue Level']

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le

# 3. Process target variables
df['recommended_workout'] = df['recommended_workout'].str.split(', ')
df['recommended_sets'] = df['recommended_sets'].str.split(', ')
df['recommended_repetitions'] = df['recommended_repetitions'].str.split(', ')

all_workouts = ['Squats', 'Lunges', 'Step-ups', 'Pushups', 'Glute Bridge',
                'Arm Circles', 'Calf Raises', 'Hammer Curl', 'Tricep Dips',
                'Bicycle Crunches', 'Russian Twist', 'Leg Raises', 'Plank', 'Crunches']

for workout in all_workouts:
    df[f'workout_{workout}'] = df['recommended_workout'].apply(lambda x: 1 if workout in x else 0)

df['recommended_sets'] = df['recommended_sets'].apply(lambda x: [int(i) for i in x])
df['recommended_repetitions'] = df['recommended_repetitions'].apply(lambda x: [int(i) for i in x])

# 4. Prepare feature and target matrices
X = df[features]
y_workouts = df[[f'workout_{w}' for w in all_workouts]]

# 5. Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_workouts, test_size=0.2, random_state=42)

# 6. Train the model with MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42,
                   activation='relu', solver='adam', learning_rate_init=0.001)
multi_target_model = MultiOutputClassifier(mlp, n_jobs=-1)
multi_target_model.fit(X_train, y_train)

# 7. Function to process derived attributes and make predictions
def recommend_workout(core_data):
    # Core attributes required: age, height, weight, fitness_goals, fitness_level, health_conditions
    user_data = core_data.copy()

    # Calculate derived attributes
    # BMI
    user_data['BMI'] = user_data['weight'] / ((user_data['height'] / 100) ** 2)

    # Preferred Workout Duration
    workout_duration_map = {'beginner': 20, 'intermediate': 40, 'advanced': 60}
    user_data['Preferred Workout Duration'] = workout_duration_map[user_data['fitness_level']]

    # Joint Issues
    joint_conditions = ['Arthritis', 'Back Pain', 'Osteoporosis', 'Rheumatoid Arthritis',
                       'Osteoarthritis', 'Fibromyalgia']
    user_data['Joint Issues'] = 'Yes' if user_data['health_conditions'] in joint_conditions else 'No'

    # Mobility & Flexibility Score
    mobility_map = {'beginner': 4, 'intermediate': 6, 'advanced': 8}
    user_data['Mobility & Flexibility Score'] = mobility_map[user_data['fitness_level']]

    # Daily Activity Type
    activity_map = {'beginner': 'Sedentary', 'intermediate': 'Active', 'advanced': 'Athlete'}
    user_data['Daily Activity Type'] = activity_map[user_data['fitness_level']]

    # Favorite Workout Types
    workout_type_map = {'lose weight': 'Cardio', 'build muscle': 'Strength', 'maintain': 'Yoga'}
    user_data['Favorite Workout Types'] = workout_type_map[user_data['fitness_goals']]

    # Fatigue Level
    if user_data['fitness_level'] == 'beginner' or user_data['age'] > 60:
        user_data['Fatigue Level'] = 'High'
    elif user_data['fitness_level'] == 'intermediate' or (user_data['Daily Activity Type'] == 'Sedentary' and user_data['age'] > 40):
        user_data['Fatigue Level'] = 'Medium'
    else:
        user_data['Fatigue Level'] = 'Low'

    # Convert to DataFrame for prediction
    user_df = pd.DataFrame([user_data])

    # Encode categorical variables
    for col in categorical_cols:
        if col in user_data:
            try:
                user_df[col] = le_dict[col].transform([user_data[col]])
            except ValueError:
                if col == 'health_conditions' and user_data[col] not in le_dict[col].classes_:
                    user_df[col] = le_dict[col].transform(['None'])
                else:
                    user_df[col] = le_dict[col].transform([le_dict[col].classes_[0]])

    # Make prediction
    workout_pred = multi_target_model.predict(user_df)[0]
    recommended_workouts = [all_workouts[i] for i, pred in enumerate(workout_pred) if pred == 1]

    # Adjust based on derived attributes
    # BMI > 30: avoid high-impact and add cardio
    if user_data['BMI'] > 30:
        high_impact = ['Squats', 'Lunges', 'Step-ups', 'Pushups']
        recommended_workouts = [w for w in recommended_workouts if w not in high_impact]
        if 'Plank' not in recommended_workouts:  # Adding a cardio-friendly exercise
            recommended_workouts.append('Plank')

    # Cap exercises based on Preferred Workout Duration
    max_exercises = {20: 3, 40: 4, 60: 5}
    recommended_workouts = recommended_workouts[:max_exercises[user_data['Preferred Workout Duration']]]

    # Ensure at least one exercise matches Favorite Workout Types
    type_exercises = {
        'Cardio': ['Plank', 'Bicycle Crunches', 'Russian Twist', 'Step-ups'],
        'Strength': ['Squats', 'Lunges', 'Pushups', 'Hammer Curl', 'Tricep Dips'],
        'Yoga': ['Glute Bridge', 'Arm Circles', 'Leg Raises', 'Calf Raises']
    }
    if not any(w in type_exercises[user_data['Favorite Workout Types']] for w in recommended_workouts):
        recommended_workouts.append(type_exercises[user_data['Favorite Workout Types']][0])

    # Sets and reps based on fitness level
    if user_data['fitness_level'] == 'beginner':
        sets = [1] * len(recommended_workouts)
        reps = [min(8 + i*2, 12) for i in range(len(recommended_workouts))]
    elif user_data['fitness_level'] == 'intermediate':
        sets = [2] * len(recommended_workouts)
        reps = [min(10 + i*2, 15) for i in range(len(recommended_workouts))]
    else:  # advanced
        sets = [3] * len(recommended_workouts)
        reps = [min(12 + i*2, 20) for i in range(len(recommended_workouts))]

    # Adjust based on Fatigue Level
    if user_data['Fatigue Level'] == 'High':
        recommended_workouts = recommended_workouts[:4]  # Cap at 4 exercises
        sets = [max(1, s-1) for s in sets][:len(recommended_workouts)]
        reps = [max(5, r-2) for r in reps][:len(recommended_workouts)]
    elif user_data['Fatigue Level'] == 'Medium':
        reps = [max(5, r-1) for r in reps]

    # Adjust for Joint Issues
    if user_data['Joint Issues'] == 'Yes':
        high_impact = ['Squats', 'Lunges', 'Step-ups', 'Pushups']
        recommended_workouts = [w for w in recommended_workouts if w not in high_impact]
        sets = sets[:len(recommended_workouts)]
        reps = reps[:len(recommended_workouts)]

    # Add flexibility exercises if Mobility & Flexibility Score < 5
    if user_data['Mobility & Flexibility Score'] < 5:
        flexibility_exercises = ['Glute Bridge', 'Arm Circles', 'Leg Raises']
        for flex_ex in flexibility_exercises:
            if flex_ex not in recommended_workouts and len(recommended_workouts) < max_exercises[user_data['Preferred Workout Duration']]:
                recommended_workouts.append(flex_ex)
                sets.append(sets[0])  # Match sets of first exercise
                reps.append(reps[0])  # Match reps of first exercise

    return {
        'recommended_workouts': recommended_workouts,
        'sets': sets,
        'repetitions': reps
    }

# Example usage with core attributes only
core_user_data = {
    'age': 35,
    'height': 170,
    'weight': 80,
    'fitness_goals': 'maintain',
    'fitness_level': 'intermediate',
    'health_conditions': 'None'
}

recommendation = recommend_workout(core_user_data)
print("Recommended Workout Plan:")
print(f"Workouts: {recommendation['recommended_workouts']}")
print(f"Sets: {recommendation['sets']}")
print(f"Repetitions: {recommendation['repetitions']}")

Recommended Workout Plan:
Workouts: ['Squats', 'Step-ups', 'Pushups', 'Glute Bridge']
Sets: [2, 2, 2, 2]
Repetitions: [9, 11, 13, 14]


In [3]:
import pickle

# Save the trained MLP model to a file
with open('workout_recommendation_mlp_model.pkl', 'wb') as file:
    pickle.dump(multi_target_model, file)

# Save the LabelEncoder dictionary (for categorical encoding)
with open('label_encoders_mlp.pkl', 'wb') as file:
    pickle.dump(le_dict, file)



In [5]:
import pandas as pd
import pickle

# Load the saved model and LabelEncoders
with open('workout_recommendation_mlp_model.pkl', 'rb') as file:
    multi_target_model = pickle.load(file)

with open('label_encoders_mlp.pkl', 'rb') as file:
    le_dict = pickle.load(file)

# Define the list of all possible workouts
all_workouts = ['Squats', 'Lunges', 'Step-ups', 'Pushups', 'Glute Bridge',
                'Arm Circles', 'Calf Raises', 'Hammer Curl', 'Tricep Dips',
                'Bicycle Crunches', 'Russian Twist', 'Leg Raises', 'Plank', 'Crunches']

# Define categorical columns for encoding
categorical_cols = ['fitness_goals', 'fitness_level', 'health_conditions',
                   'Joint Issues', 'Daily Activity Type', 'Favorite Workout Types',
                   'Fatigue Level']

# Function to process derived attributes and make predictions
def recommend_workout(core_data):
    user_data = core_data.copy()

    # Calculate derived attributes
    user_data['BMI'] = user_data['weight'] / ((user_data['height'] / 100) ** 2)
    workout_duration_map = {'beginner': 20, 'intermediate': 40, 'advanced': 60}
    user_data['Preferred Workout Duration'] = workout_duration_map[user_data['fitness_level']]
    joint_conditions = ['Arthritis', 'Back Pain', 'Osteoporosis', 'Rheumatoid Arthritis',
                       'Osteoarthritis', 'Fibromyalgia']
    user_data['Joint Issues'] = 'Yes' if user_data['health_conditions'] in joint_conditions else 'No'
    mobility_map = {'beginner': 4, 'intermediate': 6, 'advanced': 8}
    user_data['Mobility & Flexibility Score'] = mobility_map[user_data['fitness_level']]
    activity_map = {'beginner': 'Sedentary', 'intermediate': 'Active', 'advanced': 'Athlete'}
    user_data['Daily Activity Type'] = activity_map[user_data['fitness_level']]
    workout_type_map = {'lose weight': 'Cardio', 'build muscle': 'Strength', 'maintain': 'Yoga'}
    user_data['Favorite Workout Types'] = workout_type_map[user_data['fitness_goals']]
    if user_data['fitness_level'] == 'beginner' or user_data['age'] > 60:
        user_data['Fatigue Level'] = 'High'
    elif user_data['fitness_level'] == 'intermediate' or (user_data['Daily Activity Type'] == 'Sedentary' and user_data['age'] > 40):
        user_data['Fatigue Level'] = 'Medium'
    else:
        user_data['Fatigue Level'] = 'Low'

    # Convert to DataFrame for prediction
    user_df = pd.DataFrame([user_data])

    # Encode categorical variables
    for col in categorical_cols:
        if col in user_data:
            try:
                user_df[col] = le_dict[col].transform([user_data[col]])
            except ValueError:
                if col == 'health_conditions' and user_data[col] not in le_dict[col].classes_:
                    user_df[col] = le_dict[col].transform(['None'])
                else:
                    user_df[col] = le_dict[col].transform([le_dict[col].classes_[0]])

    # Make prediction
    workout_pred = multi_target_model.predict(user_df)[0]
    recommended_workouts = [all_workouts[i] for i, pred in enumerate(workout_pred) if pred == 1]

    # Adjust based on derived attributes
    if user_data['BMI'] > 30:
        high_impact = ['Squats', 'Lunges', 'Step-ups', 'Pushups']
        recommended_workouts = [w for w in recommended_workouts if w not in high_impact]
        if 'Plank' not in recommended_workouts:
            recommended_workouts.append('Plank')

    max_exercises = {20: 3, 40: 4, 60: 5}
    recommended_workouts = recommended_workouts[:max_exercises[user_data['Preferred Workout Duration']]]

    type_exercises = {
        'Cardio': ['Plank', 'Bicycle Crunches', 'Russian Twist', 'Step-ups'],
        'Strength': ['Squats', 'Lunges', 'Pushups', 'Hammer Curl', 'Tricep Dips'],
        'Yoga': ['Glute Bridge', 'Arm Circles', 'Leg Raises', 'Calf Raises']
    }
    if not any(w in type_exercises[user_data['Favorite Workout Types']] for w in recommended_workouts):
        recommended_workouts.append(type_exercises[user_data['Favorite Workout Types']][0])

    if user_data['fitness_level'] == 'beginner':
        sets = [1] * len(recommended_workouts)
        reps = [min(8 + i*2, 12) for i in range(len(recommended_workouts))]
    elif user_data['fitness_level'] == 'intermediate':
        sets = [2] * len(recommended_workouts)
        reps = [min(10 + i*2, 15) for i in range(len(recommended_workouts))]
    else:
        sets = [3] * len(recommended_workouts)
        reps = [min(12 + i*2, 20) for i in range(len(recommended_workouts))]

    if user_data['Fatigue Level'] == 'High':
        recommended_workouts = recommended_workouts[:4]
        sets = [max(1, s-1) for s in sets][:len(recommended_workouts)]
        reps = [max(5, r-2) for r in reps][:len(recommended_workouts)]
    elif user_data['Fatigue Level'] == 'Medium':
        reps = [max(5, r-1) for r in reps]

    if user_data['Joint Issues'] == 'Yes':
        high_impact = ['Squats', 'Lunges', 'Step-ups', 'Pushups']
        recommended_workouts = [w for w in recommended_workouts if w not in high_impact]
        sets = sets[:len(recommended_workouts)]
        reps = reps[:len(recommended_workouts)]

    if user_data['Mobility & Flexibility Score'] < 5:
        flexibility_exercises = ['Glute Bridge', 'Arm Circles', 'Leg Raises']
        for flex_ex in flexibility_exercises:
            if flex_ex not in recommended_workouts and len(recommended_workouts) < max_exercises[user_data['Preferred Workout Duration']]:
                recommended_workouts.append(flex_ex)
                sets.append(sets[0])
                reps.append(reps[0])

    return {
        'recommended_workouts': recommended_workouts,
        'sets': sets,
        'repetitions': reps
    }

# Function to get user input from prompt
def get_user_input():
    print("Enter your details for workout recommendation:")
    core_data = {}

    core_data['age'] = int(input("Age (e.g., 35): "))
    core_data['height'] = float(input("Height in cm (e.g., 170): "))
    core_data['weight'] = float(input("Weight in kg (e.g., 80): "))

    print("Fitness Goals (choose: 'lose weight', 'build muscle', 'maintain')")
    core_data['fitness_goals'] = input("Your fitness goal: ").lower()
    while core_data['fitness_goals'] not in ['lose weight', 'build muscle', 'maintain']:
        print("Invalid input. Choose: 'lose weight', 'build muscle', 'maintain'")
        core_data['fitness_goals'] = input("Your fitness goal: ").lower()

    print("Fitness Level (choose: 'beginner', 'intermediate', 'advanced')")
    core_data['fitness_level'] = input("Your fitness level: ").lower()
    while core_data['fitness_level'] not in ['beginner', 'intermediate', 'advanced']:
        print("Invalid input. Choose: 'beginner', 'intermediate', 'advanced'")
        core_data['fitness_level'] = input("Your fitness level: ").lower()

    print("Health Conditions (e.g., 'None', 'Arthritis', 'Back Pain', etc.)")
    core_data['health_conditions'] = input("Your health condition: ").capitalize()

    return core_data

# Main execution
if __name__ == "__main__":
    # Get user input
    user_input = get_user_input()

    # Generate recommendation
    recommendation = recommend_workout(user_input)

    # Display results in the requested format with original case
    print("\nRecommended Workouts are:")
    for i, (workout, sets, reps) in enumerate(zip(recommendation['recommended_workouts'],
                                                 recommendation['sets'],
                                                 recommendation['repetitions']), 1):
        print(f"{i}. {workout}")
        print(f"sets: {sets}")
        print(f"repetitions: {reps}\n")

Enter your details for workout recommendation:
Age (e.g., 35): 43
Height in cm (e.g., 170): 156
Weight in kg (e.g., 80): 83
Fitness Goals (choose: 'lose weight', 'build muscle', 'maintain')
Your fitness goal: maintain
Fitness Level (choose: 'beginner', 'intermediate', 'advanced')
Your fitness level: beginner
Health Conditions (e.g., 'None', 'Arthritis', 'Back Pain', etc.)
Your health condition: Diabetes Type 1

Recommended Workouts are:
1. Glute Bridge
sets: 1
repetitions: 6

2. Plank
sets: 1
repetitions: 8

3. Arm Circles
sets: 1
repetitions: 6

